# Scraping Maryland Business Licenses with Selenium

Maryland has a [great portal](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp) for searching business licenses, but the only problem is you have to check a box in order to get in.

1. Try to visit [the public search page](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp)
2. Get redirected to a "I agree to this" page. Click that you've read the disclaimer, click Enter the Site.
3. Click "Search License Records" down at the bottom of the page
4. You're now on the search page! From the "Jurisdiction" dropdown, select "Statewide"
5. In the "Trade Name" field, type "Vap%" to try to find vape shops
6. Click "Next" in the bottom right-hand corner to go to the next page
7. Click "Click for detail" to see the details for a specific business license.

That's a lot of stuff! **Let's get to work.**

## Import what you need

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

chrome_options = webdriver.ChromeOptions()

driver = webdriver.Chrome(chrome_options=chrome_options)

import pandas as pd
pd.set_option('display.max_colwidth', -1)

## Preparation

### When you search for a business license, what URL should Selenium try to visit first?

In [2]:
driver.get('https://jportal.mdcourts.gov/license/index_disclaimer.jsp')

In [3]:
button = driver.find_element_by_id('checkbox')
driver.execute_script("arguments[0].scrollIntoView(true)", button)
button.click()


**It isn't going to work, though! It's going to redirect to that intro page.** You can use *Incognito mode* to go back through the "Check the box, etc" series of pages, or you can close and re-open Chrome.

- Check the checkbox, then submit the form to accept their terms of service

Selenium can submit forms by either

- Selecting the form and using `.submit()`, or
- Selecting the button and using `.click()`

You only need to be able to get **one, not both.**

- *TIP: if something doesn't have anything special about it, xpath might be your best bet*

In [4]:
search_button = driver.find_element_by_xpath('/html/body/table/tbody/tr[7]/td/form/div/input[2]')
search_button.click()

Now click the **Search License Records** link up top in the navigation to get to the search page.

- *TIP: Honestly you could also just visit the URL directly now since you filled out that terms of service thing*

In [5]:
driver.get('https://jportal.mdcourts.gov/license/pbPublicSearch.jsp')

### Perform your search

Pick "Statewide" for the jurisdiction dropdown, and `VAP%` into the Trade Name field. The `%` is a wildcard.

In [6]:
from selenium.webdriver.support.ui import Select
select = Select(driver.find_element_by_id('slcJurisdiction'))
select.select_by_visible_text('Statewide')

In [7]:
driver.find_element_by_id('txtTradeName').send_keys('VAP%')

And now, of course, **submit the form**.

- *TIP: Since scrolling to buttons can be a pain, sometimes it's easier to select the form and use `.submit()` instead of `.click()`ing the button*

In [8]:
driver.find_element_by_tag_name('input').submit()

## (Try to) scrape the results

Let's start by just **printing this stuff**. We'll save it as a dataframe later on.

For now, just scrape **each store's name**, then cry a little. Fact: this is an impossible and miserable page. 

In [9]:
#results=driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]')
results=driver.find_elements_by_class_name('searchlistitem')
for result in results:
    print(result.text)
    

VAPE IT STORE II
VAPE IT STORE I
VAPEPAD THE
VAPE FROG
VAPE FROG


In [10]:
results=driver.find_elements_by_class_name('searchlistitem')
for result in results:
    name=result.find_element_by_class_name('copybold')
    print(name.text)

VAPE IT STORE II
VAPE IT STORE I
VAPEPAD THE
VAPE FROG
VAPE FROG


To avoid struggling with the search results page, we're going to use the **detail page** instead. Try to figure out how to select it and click it inside of your `for` loop.

- *TIP: Instead of just looking for an `a` or an `img`, you might want to look for one of its parents first, then click. This might affect the way you print out the shop's name, too*
- *TIP: Not all of them have links! You can wrap in try/except to skip it, or you can check to see if the shop's status is Pending.*

In [ ]:
# results=driver.find_elements_by_class_name('searchfieldtitle')
# for result in results:
#     try:
#         search_button = result.find_element_by_tag_name('a')
#         search_button.click() 
        
#     except:
#         print('Licence Pending Status')



Okay, now let's get to action. For each result, **click the link to the detail page** and print out the following information:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

If it doesn't have a detail page, just print out the name and that's all we need.

- *TIP: When you're done getting the information, you probably want to click back to the search results*
- *TIP: You might enjoy `find_element_by_partial_link_text` to do that*
- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*

> **IMPORTANT NOTE:** This is doomed. It's useful to do, but your current process is doomed. Once you get a `stale element reference` error move on to the next cell.

### Stale message reference

Once you navigate away from a page, and you go back to it, you can't use the variables from the first time you were on the page. So, we got a list of results when we first visited, clicked to the details page, clicked back, and now our original list is "stale."

This is sad.

Let's try this again: loop through the results and create a dataframe with `name` and `url` columns. And yes, some of them won't have URLs.

In [ ]:

results=driver.find_elements_by_class_name('searchfieldtitle')
df_vape=[]

for result in results:
    vap={}
    company=result.find_element_by_class_name('copybold')
    vap['name']=company.text
    try:
        url={}
        search_button = result.find_element_by_tag_name('a')
        vap['url']=search_button.get_attribute('href')
        
    except:
        print('no url')
    df_vape.append(vap)

In [ ]:
df=pd.DataFrame(df_vape)
df.head()

### Getting all of the results

After you've looped through the results on one page, we're going to want to go to the next page! Add a line to make it click the 'Next' button down at the bottom

- *Tip: `find_element_by_partial_link_text` will be your friend*
- *Tip: You might need to do the scrolling thing to get it onto the screen (and by that I mean, you WILL need to, so you should)*

Confirm that it moves to the next page (it doesn't need to scrape anything yet)

In [11]:
df_vape=[]

try:
    while True:
    
        results=driver.find_elements_by_class_name('searchfieldtitle')

        for result in results:

            vap={}
            company=result.find_element_by_class_name('searchlistitem')
            vap['name']=company.text
            print(company.text)

            try:
                url={}
                search_button = result.find_element_by_tag_name('a')
                vap['url']=search_button.get_attribute('href')
                print('url is done')

            except:
                pass

            df_vape.append(vap)
            
        button = driver.find_element_by_partial_link_text('Next')
        driver.execute_script("arguments[0].scrollIntoView(true)", button)
        button.click()

        print('scraping next page') 
        
            
except:
    print('done')



VAPE IT STORE II
url is done
VAPE IT STORE I
url is done
VAPEPAD THE
url is done
VAPE FROG
VAPE FROG
scraping next page
VAPE LOFT THE
url is done
VAPE N CIGAR
url is done
VAPE DOJO
url is done
VAPE HAVEN
VAPORS LOUNGE
scraping next page
VAPORFI SOUTHSIDE
url is done
VAPE DO JO
url is done
VAPERTETMD
VAPERTEKMD
url is done
VAPORIOT
scraping next page
VAPE PLUS
url is done
VAPE SOCIAL
url is done
VAPE JUNGLE
url is done
VAPE TIME
VAPEBAR EXPESS
scraping next page
VAPEWERKS
url is done
VAPIST MD
url is done
VAPOR 99
VAPOR ALLEY
url is done
VAPOR EMPORIUM
url is done
scraping next page
VAPOR VILLA
url is done
VAPOR VILLA
url is done
VAPORS LOUNGE
url is done
VAPE DOJO
url is done
VAPE DOJO
url is done
scraping next page
VAPE DOJO
url is done
VAPE DOJO
url is done
VAPEZ YARDHOUSE
VAPERS RING
url is done
done


### Wrapping with `while`

> Go back to the first page of results before you try to run this

You have a bunch of scraping code. It clicks the next button, then it stops. But you'd like it to go back up to the top! You can make that happen with a special `while` loop.

```python
while True:
    # Scrape your stuff
    # Click next button
```

This will go on FOREVER AND EVER until there is an error (when it can't find the Next button on the last page of results, you'll get an error).

- *Tip: Print out "Scraping a new page" every time you visit a new page, just to check that it's working*

In [ ]:
#check above cell

### Making it perfect

> Go back to the first page of results before you try to run this

Wrap all of your code in a `try`/`except` so that it doesn't finish with an error and you'll be good to go.

**Confirm your list has all of the vape shops in it.** If not, check where you are creating your empty list (`[]`) - if you do it in the wrong spot, it will overwrite your list every time you visit a page.

In [ ]:
#check above cell

### Save this data as a csv

The filename should be `vape-shops-basic.csv`.

In [12]:
df=pd.DataFrame(df_vape)

In [13]:
df.head()

,name,url
0,VAPE IT STORE II,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=aOVYHNAhz6w%3D
1,VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=5wlR%2B5r3t4E%3D
2,VAPEPAD THE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=fbp%2BT2lzjRM%3D
3,VAPE FROG,NaN
4,VAPE FROG,NaN


# Okay, let's scrape!

All right, get the actual data!

### Look at the URL of your first row

- *TIP: Remember `pd.set_option('display.max_colwidth', -1)` will let you see alllll of your strings*

In [14]:
pd.set_option('display.max_colwidth', -1)
df.url.head()

0    https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=aOVYHNAhz6w%3D  
1    https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=5wlR%2B5r3t4E%3D
2    https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=fbp%2BT2lzjRM%3D
3    NaN                                                                          
4    NaN                                                                          
Name: url, dtype: object

### Use Selenium to visit that page

In [ ]:
vape_data={}
driver.get('https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=jwImXHvDAeg%3D')
vape_data['address']=driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[1]').text
vape_data['location_address']=driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td').text
vape_data['licence_information']=driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[2]').text
vape_data['amount_paid']=driver.find_element_by_css_selector('body > table:nth-child(2) > tbody > tr:nth-child(4) > td:nth-child(2) > table:nth-child(8) > tbody > tr:nth-child(8) > td:nth-child(2)').text
#vape_data['amount_paid']=driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[2]/tbody/tr[8]/td[2]').text
vape_data['issued_by']=driver.find_element_by_css_selector('body > table:nth-child(2) > tbody > tr:nth-child(4) > td:nth-child(2) > table:nth-child(9) > tbody > tr.tablecelltext > td').text
vape_data

In [ ]:
driver.get('https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=jwImXHvDAeg%3D')
issued_by=driver.find_elements_by_class_name('tablecelltext')[8].text
print(issued_by)

### Now, just like you did before, grab the additional data

You should probably save it into a dictionary! Don't try to put it into the dataframe yet, though. You want:

- Mailing address
- Location address
- License information (you can keep it as one field)
- Total amount paid
- Issued by
- If you're feeling crazy, get the licenses, too.

.

- *TIP: Licenses can be acquired by doing some really odd list slicing - think about where it starts and where it ends, relative to the beginning and end of everything.*
- *TIP: If you've gotten addicted to xpath, total amount paid and issued by might not work with it when doing other shops. You'll want to test it!*

### Move all of this into one cell

It should visit the URL, then grab the data and put it into a dictionary.

### Change it into a function

You'll want to have this function accept a `row`, and send back a `pd.Series`. You can just use `pd.Series(your_dictionary)` (but it better have a better name than `your_dictionary`!).

- *TIP: Make sure you `return` something!*
- *TIP: Make sure you change everything to reflect the row's url, not the URL you typed in*

In [37]:
def vape_shops_check(row): 
    try:
        driver.get(row['url'])


        address=driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[1]').text
        location_address=driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[5]/td').text
        licence_information=driver.find_element_by_xpath('/html/body/table[2]/tbody/tr[4]/td[2]/table[1]/tbody/tr[3]/td[2]').text
        amount_paid=driver.find_elements_by_class_name('rt')[-1].text
        issued_by=driver.find_elements_by_class_name('tablecelltext')[-2].text


        return pd.Series({
            'address': address,
            'location_address': location_address,
            'licence_information': licence_information,
            'amount_paid': amount_paid,
            'issued_by': issued_by
    
    })
    except:
        print('no url')

### Use your dataframe and `.apply` to pull all of the data from the vape shops

Once you know it's working, use the whole 

- *TIP: Try using it with `.head(3)` first*
- *TIP: You'll want to use `.apply` with your new function*
- *TIP: Issued By and Total Paid are goign to give you problems if you tried to use xpath! Try checking the classes and think about find_elementSSSSS and working backwards instead of forwards.*
- *TIP: You might need a `try`/`except`*
- *TIP: Make sure you're using `axis=1`*
- *TIP: Use `.join` the big thing with all of the `dfs` - make sure you name them right!*

In [41]:
new_vape_df=adf.apply(vape_shops_check, axis=1).join(df)


no url
no url
no url
no url
no url
no url
no url
no url
no url
no url


## Save as `vape-total.csv`

Make sure you don't save the index! Open it up in a text editor or Excel to make sure it's correct.

In [42]:
new_vape_df.head()

,address,location_address,licence_information,amount_paid,issued_by,name,url
28,"WALKER TRADING CO INC\n11600 CROSSROADS CIRCLE\nSUITE E\nMIDDLE RIVER, MD 21220","VAPE DOJO\n138 N BOND STREET\nBEL AIR, MD 21014",License Status: Issued\nLicense No.: 12341351\nControl No.: 12958340\nDate of Issue: 5/29/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 12 Bel Air,$ 68.50,"JAMES J. REILLY, CLERK OF CIRCUIT COURT\n20 WEST COURTLAND STREET\nBEL AIR, MARYLAND 21014",VAPE DOJO,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=%2BSr%2FR5vYbw0%3D
33,"YEAGER ONE INC\n545 HIGGINS DR\nODENTON, MD 21113","VAPERS RING\n545 BENFIELD RD\nSEVERNA PARK, MD 21146",License Status: Issued\nLicense No.: 02311125\nControl No.: 02111865\nDate of Issue: 5/22/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 02 Anne Arundel County,$ 116.00,"ROBERT P. DUCKWORTH, CLERK OF CIRCUIT COURT\n8 CHURCH CIRCLE, ROOM H-101\nANNAPOLIS, MARYLAND 21401",VAPERS RING,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=%2BWc8uZas%2Fc8%3D
1,"AMIN NARGIS\n1104 PLANTERS PLACE\nSALISBURY, MD 21804","VAPE IT STORE I\n1724 N SALISBURY BLVD UNIT 2\nSALISBURY, MD 21801",License Status: Issued\nLicense No.: 22375605\nControl No.: 22591855\nDate of Issue: 4/27/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 22 Salisbury,$ 24.00,"WICOMICO COUNTY, CLERK OF CIRCUIT COURT\nP.O. BOX 198\nSALISBURY, MARYLAND 21803-0198",VAPE IT STORE I,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=5wlR%2B5r3t4E%3D
11,MONROE LLC,"VAPE DO JO\n2021 LIBERTY ROAD\nSYKESVILLE, MD 21784",License Status: Issued\nLicense No.: 06328812\nControl No.: 06328812\nDate of Issue: 5/03/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 06 Sykesville,$ 154.00,"DONALD B. SEALING II, CLERK OF CIRCUIT COURT\n55 NORTH COURT STREET\nWESTMINSTER, MARYLAND 21157-5155",VAPE DO JO,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=7fmfXdmjQn0%3D
16,VAPE SOCIAL LLC,"VAPE SOCIAL\n16521 S FREDERICK AVE\nGAITHERSBURG, MD 20877",License Status: Issued\nLicense No.: 15356789\nControl No.: 15728050\nDate of Issue: 4/12/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 15 Gaithersburg,$ 32.00,"BARBARA H. MEIKLEJOHN, CLERK OF CIRCUIT COURT\n50 MARYLAND AVENUE, ROOM 1300\nROCKVILLE, MARYLAND 20850",VAPE SOCIAL,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=83t6kuZ7hz8%3D
23,"VAPOR ALLEY LLC\n8117 OCEAN GATEWAY UNIT #4\nEASTON, MARYLAND 21601","VAPOR ALLEY\n8117 OCEAN GATEWAY UNIT #4\nEASTON, MD 21601",License Status: Issued\nLicense No.: 20372618\nControl No.: 20739879\nDate of Issue: 4/24/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 20 Easton,$ 229.00,"MARY ANN SHORTALL, CLERK OF CIRCUIT COURT\n11 NORTH WASHINGTON ST., STE 16\nEASTON, MARYLAND 21601",VAPOR ALLEY,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=B2EGQ9NFbzM%3D
27,VAPORS LOUNGE LLC,"VAPORS LOUNGE\n28943 THREE NOTCH ROAD\nMECHANICSVILLE, MD 20659",License Status: Issued\nLicense No.: 18369505\nControl No.: 18877819\nDate of Issue: 4/17/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 18 St. Mary's County,$ 94.00,"TRACY CANTRELL, ACTING CLERK OF CIRCUIT COURT\n41605 COURTHOUSE DRIVE, PO BOX 676\nLEONARDTOWN, MARYLAND 20650",VAPORS LOUNGE,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=BXFMn5hM1C8%3D
6,DISCOUNT TOBACCO ESSEX LLC,"VAPE N CIGAR\n7104 MINSTREL UNIT #7\nCOLUMBIA, MD 21045",License Status: Issued\nLicense No.: 13343011\nControl No.: 13856368\nDate of Issue: 4/30/2018\nMonths Paid: 12\nExp. Date: 4/30/2019\nSubdivision: 13 Howard County,$ 84.00,"WAYNE A. ROBEY, CLERK OF CIRCUIT COURT\n9250 BENDIX ROAD\nCOLUMBIA, MARYLAND 21045",VAPE N CIGAR,https://jportal.mdcourts.gov/license/pbLicenseDetail.jsp?owi=DhUw%2BdoofGI%3D
15,"TRUE ENTERPRISE INC\n5967 OXBRIDGE DRIVE\nSALISBURY, MD 21801","VAPE PLUS\n519 LINDEN AVENUE UNIT 6\nPOCOMOKE CITY, MD 21851",License Status: Issued\nLicense No.: 23380005\nControl No.: 23990202\nDate of 

In [43]:
new_vape_df.to_csv("vape-total.csv", index=False)